Two layer CNN is run on small dataset consisting of 20 images, 5 images per category. Input has two channels, log image and original image. Network did not overfit on this data and the loss is constant even after 200 epochs.

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import glob
import csv
from torch.utils import data

import matplotlib.image as mpimg
seed = 7
np.random.seed(seed)
torch.manual_seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose(
    [transforms.ToTensor()])

class Dataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, data_path, transform):
        'Initialization'
        self.data_path = data_path
        self.transform = transform
        X_apples = []
        X_dates = []
        X_tyre = []
        X_paper = []
        self.X = []
        self.Y = []

        for image_path in glob.glob(self.data_path + 'Apples/img*/csv/log/*.csv'):
            log = np.array(list(csv.reader(open(image_path, "r"), delimiter=","))).astype("double")
            original_image_path = image_path.replace("log","original")
            original = np.array(list(csv.reader(open(original_image_path, "r"), delimiter=","))).astype("double")
            channels = []
            channels.append(log)
            channels.append(original)
            
            channels = np.asarray(channels,"double")
            X_apples.append(channels)
            
        X_apples = np.asarray(X_apples, "double")
        
        # #assign labels
        Y_apples = np.zeros(X_apples.shape[0], dtype=np.long)

        for image_path in glob.glob(self.data_path + 'Dates/img*/csv/log/*.csv'):
            log = np.array(list(csv.reader(open(image_path, "r"), delimiter=","))).astype("double")
            original_image_path = image_path.replace("log","original")
            original = np.array(list(csv.reader(open(original_image_path, "r"), delimiter=","))).astype("double")
            channels = []
            channels.append(log)
            channels.append(original)
            
            channels = np.asarray(channels,"double")
            X_dates.append(channels)
            
        X_dates = np.asarray(X_dates, "double")
        
        # #assign labels
        Y_dates = np.ones(X_dates.shape[0], dtype=np.long)

        for image_path in glob.glob(self.data_path + 'TYRE/img*/csv/log/*.csv'):
            log = np.array(list(csv.reader(open(image_path, "r"), delimiter=","))).astype("double")
            original_image_path = image_path.replace("log","original")
            original = np.array(list(csv.reader(open(original_image_path, "r"), delimiter=","))).astype("double")
            channels = []
            channels.append(log)
            channels.append(original)
           
            channels = np.asarray(channels,"double")
            X_tyre.append(channels)
           
        X_tyre = np.asarray(X_tyre, "double")
       
        # #assign labels
        Y_tyre = 2 * np.ones(X_tyre.shape[0], dtype=np.long)

        for image_path in glob.glob(self.data_path + 'Paper/img*/csv/log/*.csv'):
            log = np.array(list(csv.reader(open(image_path, "r"), delimiter=","))).astype("double")
            original_image_path = image_path.replace("log","original")
            original = np.array(list(csv.reader(open(original_image_path, "r"), delimiter=","))).astype("double")
            channels = []
            channels.append(log)
            channels.append(original)
            
            channels = np.asarray(channels,"double")
            X_paper.append(channels)
           
        X_paper = np.asarray(X_paper, "double")
        
        # #assign labels
        Y_paper = 3 * np.ones(X_paper.shape[0], dtype=np.long)

        self.X = np.append(X_apples, X_dates, axis=0)
        self.X = np.append(self.X,X_tyre, axis=0)
        self.X = np.append(self.X,X_paper, axis=0)
        self.Y = np.append(Y_apples,Y_dates, axis=0)
        self.Y = np.append(self.Y,Y_tyre, axis=0)
        self.Y = np.append(self.Y,Y_paper, axis=0)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.Y)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        image = self.X[index]
        image = image.transpose((1, 2, 0))
        label = self.Y[index]

        if self.transform:
            image = self.transform(image)
        
        return image, label

training_set = Dataset('./data/train/', transform=transform)
training_generator = data.DataLoader(training_set, batch_size=4,
                                          shuffle=True, num_workers=2)

validation_set = Dataset('./data/test/', transform=transform)
validation_generator = data.DataLoader(validation_set, batch_size=4,
                                          shuffle=False, num_workers=2)


classes = ( 'Apples', 'Dates','TYRE', 'Paper')


import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 16, 5)
        self.fc1 = nn.Linear(16 * 13 * 13, 512)
        #self.fc2 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 13 * 13)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = self.fc2(x)
        return x


net = Net()
net.double()

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
n_mini_batch = 0
for epoch in range(200):  # loop over the dataset multiple times

    class_correct_train = list(0. for i in range(4))
    class_total_train = list(0. for i in range(4))
    running_loss = 0.0

    for i, labelled_data in enumerate(training_generator, 0):
        n_mini_batch += 1
        # get the inputs
        inputs, labels = labelled_data
        inputs = inputs.type(torch.DoubleTensor)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct_train[label] += c[i].item()
            class_total_train[label] += 1

    if(epoch % 10 == 0):
        print("Accuracy: [" + str(epoch + 1) + "] = " + str(np.sum(class_correct_train) / np.sum(class_total_train)))
        print('[%d] running_loss: %.3f loss: %.3f' % (epoch + 1, running_loss/5, loss.item()))
        running_loss = 0

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    for labelled_data in validation_generator:
        images, labels = labelled_data
        images = images.type(torch.DoubleTensor)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print ("Total accuracy: " + str(np.sum(class_correct) / np.sum(class_total)))

Accuracy: [1] = 0.25
[1] running_loss: 4284005839002123264.000 loss: 825274528011046400.000
Accuracy: [11] = 0.25
[11] running_loss: 1.397 loss: 1.405
Accuracy: [21] = 0.25
[21] running_loss: 1.387 loss: 1.380
Accuracy: [31] = 0.25
[31] running_loss: 1.387 loss: 1.386
Accuracy: [41] = 0.25
[41] running_loss: 1.387 loss: 1.398
Accuracy: [51] = 0.25
[51] running_loss: 1.386 loss: 1.381
Accuracy: [61] = 0.25
[61] running_loss: 1.387 loss: 1.389
Accuracy: [71] = 0.25
[71] running_loss: 1.387 loss: 1.384
Accuracy: [81] = 0.25
[81] running_loss: 1.387 loss: 1.381
Accuracy: [91] = 0.25
[91] running_loss: 1.387 loss: 1.385
Accuracy: [101] = 0.25
[101] running_loss: 1.387 loss: 1.385
Accuracy: [111] = 0.25
[111] running_loss: 1.386 loss: 1.386
Accuracy: [121] = 0.25
[121] running_loss: 1.386 loss: 1.386
Accuracy: [131] = 0.25
[131] running_loss: 1.386 loss: 1.386
Accuracy: [141] = 0.25
[141] running_loss: 1.386 loss: 1.387
Accuracy: [151] = 0.25
[151] running_loss: 1.387 loss: 1.387
Accuracy: [

Increasing the learning rate by factor of 10

In [7]:
net = Net()
net.double()

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in range(200):  # loop over the dataset multiple times

    class_correct_train = list(0. for i in range(4))
    class_total_train = list(0. for i in range(4))
    running_loss = 0.0

    for i, labelled_data in enumerate(training_generator, 0):
        n_mini_batch += 1
        # get the inputs
        inputs, labels = labelled_data
        inputs = inputs.type(torch.DoubleTensor)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct_train[label] += c[i].item()
            class_total_train[label] += 1

    if(epoch % 10 == 0):
        print("Accuracy: [" + str(epoch + 1) + "] = " + str(np.sum(class_correct_train) / np.sum(class_total_train)))
        print('[%d] running_loss: %.3f loss: %.3f' % (epoch + 1, running_loss/5, loss.item()))
        running_loss = 0

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    for labelled_data in validation_generator:
        images, labels = labelled_data
        images = images.type(torch.DoubleTensor)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print ("Total accuracy: " + str(np.sum(class_correct) / np.sum(class_total)))

Accuracy: [1] = 0.15
[1] running_loss: 111803715180085730857180037398921216.000 loss: 559015822954375394181857009609998336.000
Accuracy: [11] = 0.25
[11] running_loss: 1.390 loss: 1.389
Accuracy: [21] = 0.1
[21] running_loss: 1.388 loss: 1.384
Accuracy: [31] = 0.25
[31] running_loss: 1.388 loss: 1.389
Accuracy: [41] = 0.15
[41] running_loss: 1.387 loss: 1.387
Accuracy: [51] = 0.25
[51] running_loss: 1.388 loss: 1.391
Accuracy: [61] = 0.25
[61] running_loss: 1.388 loss: 1.392
Accuracy: [71] = 0.25
[71] running_loss: 1.389 loss: 1.389
Accuracy: [81] = 0.25
[81] running_loss: 1.387 loss: 1.386
Accuracy: [91] = 0.25
[91] running_loss: 1.388 loss: 1.392
Accuracy: [101] = 0.25
[101] running_loss: 1.387 loss: 1.386
Accuracy: [111] = 0.1
[111] running_loss: 1.387 loss: 1.389
Accuracy: [121] = 0.25
[121] running_loss: 1.388 loss: 1.388
Accuracy: [131] = 0.2
[131] running_loss: 1.388 loss: 1.393
Accuracy: [141] = 0.2
[141] running_loss: 1.389 loss: 1.395
Accuracy: [151] = 0.25
[151] running_loss

Further increasing learning rate:

In [9]:
net = Net()
net.double()

optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)

for epoch in range(200):  # loop over the dataset multiple times

    class_correct_train = list(0. for i in range(4))
    class_total_train = list(0. for i in range(4))
    running_loss = 0.0

    for i, labelled_data in enumerate(training_generator, 0):
        n_mini_batch += 1
        # get the inputs
        inputs, labels = labelled_data
        inputs = inputs.type(torch.DoubleTensor)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct_train[label] += c[i].item()
            class_total_train[label] += 1

    if(epoch % 10 == 0):
        print("Accuracy: [" + str(epoch + 1) + "] = " + str(np.sum(class_correct_train) / np.sum(class_total_train)))
        print('[%d] running_loss: %.3f loss: %.3f' % (epoch + 1, running_loss/5, loss.item()))
        running_loss = 0

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    for labelled_data in validation_generator:
        images, labels = labelled_data
        images = images.type(torch.DoubleTensor)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print ("Total accuracy: " + str(np.sum(class_correct) / np.sum(class_total)))

Accuracy: [1] = 0.25
[1] running_loss: 330823187869420467681708629774177073362824786345984.000 loss: 4493120137215466281146504801817063013783126933504.000
Accuracy: [11] = 0.2
[11] running_loss: 1.409 loss: 1.436
Accuracy: [21] = 0.1
[21] running_loss: 1.405 loss: 1.425
Accuracy: [31] = 0.05
[31] running_loss: 1.408 loss: 1.390
Accuracy: [41] = 0.2
[41] running_loss: 1.393 loss: 1.426
Accuracy: [51] = 0.1
[51] running_loss: 1.399 loss: 1.439
Accuracy: [61] = 0.25
[61] running_loss: 1.390 loss: 1.403
Accuracy: [71] = 0.2
[71] running_loss: 1.396 loss: 1.408
Accuracy: [81] = 0.25
[81] running_loss: 1.395 loss: 1.436
Accuracy: [91] = 0.15
[91] running_loss: 1.407 loss: 1.430
Accuracy: [101] = 0.2
[101] running_loss: 1.396 loss: 1.445
Accuracy: [111] = 0.15
[111] running_loss: 1.410 loss: 1.464
Accuracy: [121] = 0.25
[121] running_loss: 1.417 loss: 1.549
Accuracy: [131] = 0.05
[131] running_loss: 1.418 loss: 1.459
Accuracy: [141] = 0.15
[141] running_loss: 1.421 loss: 1.503
Accuracy: [151]

In [10]:
net = Net()
net.double()

optimizer = optim.SGD(net.parameters(), lr=1, momentum=0.9)

for epoch in range(200):  # loop over the dataset multiple times

    class_correct_train = list(0. for i in range(4))
    class_total_train = list(0. for i in range(4))
    running_loss = 0.0

    for i, labelled_data in enumerate(training_generator, 0):
        n_mini_batch += 1
        # get the inputs
        inputs, labels = labelled_data
        inputs = inputs.type(torch.DoubleTensor)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct_train[label] += c[i].item()
            class_total_train[label] += 1

    if(epoch % 10 == 0):
        print("Accuracy: [" + str(epoch + 1) + "] = " + str(np.sum(class_correct_train) / np.sum(class_total_train)))
        print('[%d] running_loss: %.3f loss: %.3f' % (epoch + 1, running_loss/5, loss.item()))
        running_loss = 0

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    for labelled_data in validation_generator:
        images, labels = labelled_data
        images = images.type(torch.DoubleTensor)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print ("Total accuracy: " + str(np.sum(class_correct) / np.sum(class_total)))

Accuracy: [1] = 0.15
[1] running_loss: 2029914068329814528.000 loss: 12843498.249
Accuracy: [11] = 0.25
[11] running_loss: 1.609 loss: 1.889
Accuracy: [21] = 0.25
[21] running_loss: 1.530 loss: 1.251
Accuracy: [31] = 0.25
[31] running_loss: 1.545 loss: 1.727
Accuracy: [41] = 0.2
[41] running_loss: 1.606 loss: 1.791
Accuracy: [51] = 0.0
[51] running_loss: 1.857 loss: 1.901
Accuracy: [61] = 0.4
[61] running_loss: 1.816 loss: 1.885
Accuracy: [71] = 0.15
[71] running_loss: 1.813 loss: 2.129
Accuracy: [81] = 0.15
[81] running_loss: 2.179 loss: 2.309
Accuracy: [91] = 0.0
[91] running_loss: 1.855 loss: 2.068
Accuracy: [101] = 0.2
[101] running_loss: 1.489 loss: 1.487
Accuracy: [111] = 0.25
[111] running_loss: 1.549 loss: 1.950
Accuracy: [121] = 0.25
[121] running_loss: 1.482 loss: 1.382
Accuracy: [131] = 0.1
[131] running_loss: 1.637 loss: 1.740
Accuracy: [141] = 0.3
[141] running_loss: 1.573 loss: 2.202
Accuracy: [151] = 0.15
[151] running_loss: 1.908 loss: 1.770
Accuracy: [161] = 0.15
[161]

With normalized input_channels, network is overfitting on small dataset.

In [11]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((7.78748281657, 7025.73150657), (0.854766446669, 7977.58896648))])

training_set = Dataset('./data/train/', transform=transform)
training_generator = data.DataLoader(training_set, batch_size=4,
                                          shuffle=True, num_workers=2)

validation_set = Dataset('./data/test/', transform=transform)
validation_generator = data.DataLoader(validation_set, batch_size=4,
                                          shuffle=False, num_workers=2)

net = Net()
net.double()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(200):  # loop over the dataset multiple times

    class_correct_train = list(0. for i in range(4))
    class_total_train = list(0. for i in range(4))
    running_loss = 0.0

    for i, labelled_data in enumerate(training_generator, 0):
        n_mini_batch += 1
        # get the inputs
        inputs, labels = labelled_data
        inputs = inputs.type(torch.DoubleTensor)
       
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct_train[label] += c[i].item()
            class_total_train[label] += 1

    if(epoch % 10 == 0):
        print("Accuracy: [" + str(epoch + 1) + "] = " + str(np.sum(class_correct_train) / np.sum(class_total_train)))
        print('[%d] running_loss: %.3f loss: %.3f' % (epoch + 1, running_loss/5, loss.item()))
        running_loss = 0

print('Finished Training')

class_correct = list(0. for i in range(4))
class_total = list(0. for i in range(4))
with torch.no_grad():
    for labelled_data in validation_generator:
        images, labels = labelled_data
        images = images.type(torch.DoubleTensor)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(4):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))
print ("Total accuracy: " + str(np.sum(class_correct) / np.sum(class_total)))

Accuracy: [1] = 0.35
[1] running_loss: 1.421 loss: 1.295
Accuracy: [11] = 0.95
[11] running_loss: 0.266 loss: 0.358
Accuracy: [21] = 1.0
[21] running_loss: 0.043 loss: 0.003
Accuracy: [31] = 1.0
[31] running_loss: 0.009 loss: 0.014
Accuracy: [41] = 1.0
[41] running_loss: 0.005 loss: 0.000
Accuracy: [51] = 1.0
[51] running_loss: 0.003 loss: 0.004
Accuracy: [61] = 1.0
[61] running_loss: 0.002 loss: 0.000
Accuracy: [71] = 1.0
[71] running_loss: 0.002 loss: 0.005
Accuracy: [81] = 1.0
[81] running_loss: 0.001 loss: 0.001
Accuracy: [91] = 1.0
[91] running_loss: 0.001 loss: 0.000
Accuracy: [101] = 1.0
[101] running_loss: 0.001 loss: 0.001
Accuracy: [111] = 1.0
[111] running_loss: 0.001 loss: 0.001
Accuracy: [121] = 1.0
[121] running_loss: 0.001 loss: 0.001
Accuracy: [131] = 1.0
[131] running_loss: 0.001 loss: 0.001
Accuracy: [141] = 1.0
[141] running_loss: 0.001 loss: 0.001
Accuracy: [151] = 1.0
[151] running_loss: 0.001 loss: 0.000
Accuracy: [161] = 1.0
[161] running_loss: 0.001 loss: 0.001
